# Assignment 2 - Building a decision tree

This is a skeleton of a decision tree classifier for the example data set in `data/example.csv`.

In [89]:
import csv
import  math
from statistics import median, mode, mean
from collections import Counter
from enum import Enum
import numpy as np
import scipy.stats as st


The input filename is hard-coded.

In [90]:
#trainingSet = np.loadtxt('data/housing_price_train.csv',dtype='<U20',delimiter=',')
#testSet = np.loadtxt('data/housing_price_test.csv',dtype='<U20',delimiter=',')
trainingSet = np.loadtxt('data/train.csv',dtype='<U20',delimiter=',')
testSet = np.loadtxt('data/test.csv',dtype='<U20',delimiter=',')

The attribute labels types are hard-coded too (the same order as in the file!).

In [91]:
#testAttributes = trainingSet[0,:80]
#split training set to training set 75% and test set 25%:

#testSet = trainingSet[-366:,:]
#testTarget = testSet[:,-1] #real target values
#testSet = trainingSet[-366:,:80]
#testSet = np.vstack((testAttributes,testSet)) #add attribute labels to testSet
#trainingSet = trainingSet[:1095,:]


testLabels = testSet[0]
testAttributes = testLabels[1:]
trainingLabels = trainingSet[0]
trainingData= np.transpose(np.copy(trainingSet[1:]))
testData = np.transpose(np.copy(testSet[1:]))
testData = np.copy(testData)
testAttributes =np.copy(testLabels)
housingPrices = np.log10(np.float64(trainingData[-1]))
trainingData = np.copy(trainingData[1:-1])
trainingLabels = np.copy(trainingLabels[1:-1])
dupAttrs= np.array(['3SsnPorch','Alley','MoSold','PoolQC','Utilities','YrSold','LandSlope','LotFrontage','MasVnrType',
                    'MasVnrArea','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Electrical',
                    'FireplaceQu','GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond','Fence','MiscFeature',
                    'PoolArea','Street',])
uniqueAttrs = np.array([[trainingLabels[i],i]  for i in range(len(trainingLabels)) if (trainingLabels[i] not in dupAttrs)])
uniqueLabels = uniqueAttrs[:,0]
trainData = trainingData[np.int64(uniqueAttrs[:,1])]

The index of the target attribute (assuming it's the last).

In [92]:
#IDX_TARGET = np.log10(np.float64(trainingSet[-1]))

A main class DT representing the decision tree classifier. It could represent with methods:

  - a given impurity measure;
  - the search for the best attribute to split with;
  - the addition of a node to the tree;
  - a convenient model printer;
  - the recursive call for obtaining a tree;
  - a builder and an applier.

In [93]:
def probabilities(attribute):
    ''' 
    compute probabilities (used for computation of entropy)
    '''
    uniqueVals, val_counts=np.unique(attribute,return_counts=True)
    val_counts = np.float64(val_counts)
    val_probs = val_counts/np.float64(attribute.size)
    return uniqueVals, val_probs
def entropy(attribute):
    ''' 
    compute entropy which is used for infogain
    '''
    uniqueVals , val_probs =probabilities(attribute)
    return np.sum(val_probs *np.log2(1./val_probs))
def infoGain(attribute,attributes):
    ''' 
    compute infogain for a given attribute
    '''
    attr_probs = probabilities(attribute)
    label_entropy = entropy(attributes)
    label_given_attr = []
    if type(attr_probs[1]) != np.ndarray:
        return label_entropy
    cts_dict = dict(zip(attr_probs[0],attr_probs[1]))
    for uniq_attr_val,uniq_val_prob in cts_dict.items():
        label_given_attr.append( uniq_val_prob *entropy(attributes[attribute==uniq_attr_val]))
    return label_entropy- np.sum(label_given_attr)
def find_best_attr(trainData, attribute_names,attributes):
    ''' 
    Find best attribute to split data on
    '''
    gains=np.array([])
    for i in trainData:
        gains=np.append( gains, infoGain(i, attributes))
    maxID = np.where(gains == np.max(gains))[0][0]
    maxgn = gains[maxID]
    att_name= attribute_names[maxID]
    print('Max Gain:', maxgn)
    return att_name,maxID
def stringToInt(attribute):
    ''' 
    Convert attribute from string to int
    '''
    uniqueVals, val_counts = np.unique(attribute,return_counts=True)
    map_ints = range(len(uniqueVals))
    map_attr = np.copy(attribute)
    for i in range(len(uniqueVals)):
        all_val = np.where(attribute == uniqueVals[i])[0]
        map_attr[all_val] = map_ints[i]
        intvals = map_ints[i]
        allints = np.ones_like(all_val)*intvals
    return [uniqueVals, map_ints, val_counts,map_attr]
class DT:
    def __init__(self, attr_split=None,parent=None):
        self.parent=parent
        self.child=[]
        self.attr_split= attr_split
    def add_child(self,child):
        self.child.append(child)
        child.parent= self
def isNaN(arr1,arr2):
    nans = np.where( (~np.isnan(arr1)) &(~np.isnan(arr2)))[0]
    fin_arr1 = arr1[nans]
    fin_arr2 = arr2[nans]
    return fin_arr1,fin_arr2,nans
def findBestSplit(attribute,attributes,depth=0):
    ''' 
    find best splitting point based on attribute type (numeric or categorical)
    '''
    try:
        attr_dat =np.float64(attribute)
        uniqueVals,probs = probabilities(attribute)
        typ='num'    
    except:
        uniqueVals,map_ints, val_counts, map_attr = stringToInt(attribute)
        attr_dat = np.copy(np.float64(map_attr))
        typ='cat'
    if typ =='cat' or len(uniqueVals) <=10:
        n_splits = len(uniqueVals)
        splits =[]
        split_ints = []
        for i in range(n_splits):
            splits.append([attribute==uniqueVals[i] ])
            split_ints.append(uniqueVals[i])
        threshold_val = np.array([-99])
        threshold_type ='NA'
    elif len(uniqueVals) >2:
        try:
            attributes = np.float64(attributes)
            sort_by_attr = np.argsort(attr_dat)
            attr_sort = np.copy(attr_dat[sort_by_attr])
            label_sort =np.copy(attributes[sort_by_attr])
            linearegslopes_left = []
            linearegslopes_right = []
            for i in range(len(attr_sort)//10,len(attr_sort)-len(attr_sort)//10):
                attr_l = attr_sort[0:i]
                attr_r = attr_sort[i:]
                lab_l = label_sort[0:i]
                lab_r = label_sort[i:]
                m_l,b_l = np.polyfit(attr_l,lab_l,1)
                m_r,b_r = np.polyfit(attr_r,lab_r,1)
                linearegslopes_left.append(abs(m_l))
                linearegslopes_right.append(abs(m_r))
            linearegslopes_left =np.array(linearegslopes_left)
            linearegslopes_right =np.array(linearegslopes_right)
            finite_left,finite_right,finite_inds = isNaN(linearegslopes_left,linearegslopes_right)
            inv_left = 1./finite_left
            inv_right = 1./finite_right
            linearleft=finite_left*inv_right
            linright = inv_left*finite_right
            finmxleft = np.where(linearleft == np.max(linearleft))[0][0]
            mxleft = np.where(linearegslopes_left == finite_left[finmxleft])[0]
            finmxright = np.where(linearight == np.max(linearight))[0][0]
            mxright = np.where(linearegslopes_right == finite_right[finmxright])[0]
            threshold_val = attr_sort[mxleft+len(attr_sort)//10]        
            if typ=='cat':
                split_l = [attr_dat <threshold_val]
                split_r = [attr_dat >=threshold_val]
                splits=[split_l,split_r]
            else:
                split_l = [attr_dat<=threshold_val]
                split_r= [attr_dat >threshold_val]
                splits=[split_l,split_r]
            threshold_type='linearSplit'
            split_ints =[]
        except:
            threshold_val = np.array([np.mean(attr_dat)])
            split_l =[attr_dat < threshold_val]
            split_r = [attr_dat >= threshold_val]
            splits = [split_l,split_r]
            threshold_type = 'linearSplit'
            split_ints = []
    return splits,threshold_val,threshold_type,split_ints,typ
def pure(vals):
    ''' 
    Check if vals are pure or not
    '''
    return len(np.unique(vals))==1
def buildTree(dat,attribute_names,attributes,root=None,depth=0):
    '''
    find best attribute to split on, best splitting point and build a decision tree recursively
    '''
    if dat[0] ==[]:
        root.avg_label = root.parent.avg_label
        return
    if len(dat[0]) <5:
        root.avg_label= np.mean(attributes)
        return
    bestattr,maxID= find_best_attr(dat,attribute_names,attributes)
    if not root:
        root = DT()
    root.split_attr =bestattr
    root.split_attr_ind=maxID
    attr_data = np.copy(dat[maxID])
    if pure(attr_data):
        root.avg_label= np.mean(attributes)
        return 
    splits,threshold_val, threshold_type,split_ints,typ=findBestSplit(attr_data,attributes,depth=depth)
    trees = []
    for i in range(len(splits)):
        trees.append(DT(parent=root))
        root.add_child(trees[i])
        root.split_ints = split_ints
        trees[i].inds_filt = splits[i][0]
        
    root.threshold_val =threshold_val
    root.threshold_type = threshold_type
    root.typ = typ
    root.avg_label= np.mean(attributes)
    depth+=1
    red= [attribute_names!=bestattr]

    red_attr_names = attribute_names[red]
    red_data = dat[red]
    for tree in trees:
        buildTree(red_data[:,tree.inds_filt],red_attr_names,attributes[tree.inds_filt],root=tree,depth=depth)
    return root 
price_by_id = {}
decisionT = buildTree(trainData,uniqueLabels, housingPrices)

def predict(dat,uniqueLabels,dTree,predTree = None,attributes=[]):
    ''' 
    Predict target attributes (housing prices)
    '''
    if len(dat[0]) == 0:
        return
    if len(attributes) == 0:
        attributes = np.ones_like(dat[0])
    if len(dTree.child) ==0:
        for tid in dat[0]:
            price_by_id[tid] = dTree.avg_label 
        return
    if not predTree:
        predTree = DT()
    attr_split = dTree.split_attr 
    attr_split_ind =np.where(uniqueLabels == attr_split)[0]
    attr_split_ints = dTree.split_ints
    attr_dat = dat[attr_split_ind][0]
    thresh_type = dTree.threshold_type
    thresh_val = np.float64(dTree.threshold_val[0])
    try:
        attr_data =np.float64(attr_dat)
        uniqueVals,probs = probabilities(attr_dat)
    except:
        uniqueVals,map_ints, val_counts, map_attr = stringToInt(attr_dat)
        attr_data = np.copy(np.float64(map_attr))
    typ = dTree.typ
    predTree.attr_split = attr_split
    if thresh_type=='0':
        split_l = np.where(attr_data == 0)[0]
        split_r = np.where(attr_data > 0)[0]
        splits = [split_l,split_r]
    elif thresh_type=='linearSplit':
        if typ=='cat':
            split_l = np.where(attr_data <thresh_val)[0]
            split_r = np.where(attr_data >=thresh_val)[0]
            splits=[split_l,split_r]
        else:
            split_l = np.where(attr_data<=thresh_val)[0]
            split_r= np.where(attr_data >thresh_val)[0]       
            splits=[split_l,split_r]   
    elif thresh_type == 'NA':
        splits =[]
        for val in attr_split_ints:
            splits.append(np.where(attr_data==val)[0])    
    trees = []
    for i in range(len(splits)):
        trees.append(DT(parent=predTree))
        predTree.add_child(trees[i])
        trees[i].label_inds = splits[i]
        trees[i].dtree = dTree.child[i]
        trees[i].avg_label =dTree.avg_label
        predict(dat[:,trees[i].label_inds],uniqueLabels,trees[i].dtree, predTree= trees[i],attributes=attributes)
    tids = dat[0]
    labs =[]
    for tid in tids:
        try:
            labs.append(price_by_id[tid])
        except:
            labs.append(np.median(list(price_by_id.values() )))
    return labs
p = predict(testData, testAttributes,decisionT)

def __mean_squared_error():
        """
        Calculates mean squared error for a selection of records.

        :param records: Data records (given by indices)
        """
        # TODO
        global p
        global testTarget
        SQE = 0
        testTargetFloat = [float(i) for i in testTarget]
        for i,x in enumerate(p):
            SQE = SQE + ((10**x)-float(testTarget[i]))**2
        MSE = np.sqrt(SQE/len(p))/np.std(testTargetFloat)
        return MSE

C:\Users\henri\Anaconda3\lib\site-packages\ipykernel_launcher.py:143: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


Max Gain: 8.045796138342293


C:\Users\henri\Anaconda3\lib\site-packages\ipykernel_launcher.py:173: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
C:\Users\henri\Anaconda3\lib\site-packages\ipykernel_launcher.py:174: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


Max Gain: 7.446101489756081
Max Gain: 6.524561605715065
Max Gain: 6.497662618424865
Max Gain: 5.65645677569764
Max Gain: 5.631615665225583
Max Gain: 5.8574319218048085
Max Gain: 6.595891163108825
Max Gain: 7.149981774453494
Max Gain: 7.43752996644357
Max Gain: 7.331770310806017
Max Gain: 7.21647843135994
Max Gain: 7.813184524595746
Max Gain: 7.126460214904276
Max Gain: 6.247495789386386
Max Gain: 5.94770277922009
Max Gain: 6.058984089445426
Max Gain: 6.87496155606346
Max Gain: 7.164965687925738
Max Gain: 6.457972024550777
Max Gain: 6.11249200111032
Max Gain: 5.304682449772211
Max Gain: 4.566108939837479
Max Gain: 4.436605434317882
Max Gain: 3.321928094887362
Max Gain: 3.7004397181410926
Max Gain: 6.364489555653822
Max Gain: 5.720049960644811
Max Gain: 5.155399311574898


In [94]:
import pandas as pd
def createSubmission(test_ids, predictions):
    sub = pd.DataFrame()
    sub['Id'] = test_ids
    sub['SalePrice'] = predictions
    sub.to_csv('submission.csv',index=False)

Finally, the main function building a decision tree model, printing it and applying it on some unseen records.

In [96]:
def main():
    
    test_ids = []
    predictions = []
    for i in range(len(testData[0])):
        test_ids.append(str(testData[0][i]))
        predictions.append(str(10**p[i]))
    
    createSubmission(test_ids, predictions)
    
    #https://www.kaggle.com/c/house-prices-advanced-regression-techniques/leaderboard
    print("Kaggle Test Score: 0.29785")
    
    #print("Root-Mean-Square-Error (normalized using np.std):")
    #print(__mean_squared_error())

    
    #print_tree(decision_tree)

if __name__ == "__main__":
    main()

### 